<a href="https://colab.research.google.com/github/cras-lab/ForMe/blob/main/Consistency_Index.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install -q XlsxWriter

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.5/154.5 kB 4.4 MB/s eta 0:00:00


In [29]:
from google.colab import files
import pandas as pd
import numpy as np
import math

uploaded = files.upload()
filename = list(uploaded.keys())[0]

src_path = "/content/" + filename
df = pd.read_excel(src_path)


NUM_CLASS = 5
NUM_Q = 4
NUM_PERSON = 15
NUM_COMB = math.comb( NUM_Q, 2)

def GetMatrix( col ):
  matrix = [ [1, col[0], col[1], col[2] ],
             [1/col[0], 1, col[3], col[4]],
             [1/col[1], 1/col[3], 1, col[5]],
             [ 1/col[2], 1/col[4], 1/col[5], 1]
           ]
  return matrix

def FindEigenValue( col ):
  mat = GetMatrix(col)
  eigenvalues, eigenvectors = np.linalg.eig(mat)
  principal_eigenvalue = max(abs(eigenvalues))

  return principal_eigenvalue

def FindEigenValues( col ):
  eigens = np.zeros(NUM_CLASS)

  for i in range( 0, NUM_CLASS):
    col1 = col[i*NUM_COMB:(i+1)*NUM_COMB]
    eigens[i] = FindEigenValue(col1)

  return eigens

def GetWeight(col):
  harmonic = np.zeros(NUM_Q*NUM_CLASS)
  weight = np.zeros(NUM_Q*NUM_CLASS)

  for i in range( 0, NUM_CLASS):
    col1 = col[i*NUM_COMB:(i+1)*NUM_COMB]
    mat = GetMatrix(col1)

    for j in range(0, NUM_Q):
      harmonic[i*NUM_Q+j] = (mat[j][0]*mat[j][1]*mat[j][2]*mat[j][3])**(1/NUM_Q)

    total = sum( harmonic[i*NUM_Q:i*NUM_Q+4])

    weight[i*NUM_Q:i*NUM_Q+4 ] = [ x/total for x in harmonic[i*NUM_Q:i*NUM_Q+4]]

  return weight


e_values = np.zeros( (NUM_PERSON+1, NUM_CLASS) )


CI = np.zeros( (NUM_PERSON+1, NUM_CLASS) )

Wgt = np.zeros( (NUM_PERSON+1, NUM_CLASS*NUM_Q ) )

for i in range(1, NUM_PERSON+2):
    col = df[ df.columns[i]].values
    e_values[i-1] = FindEigenValues( col )
    CI[i-1] = [ (x-4)/3/0.9 for x in e_values[i-1] ]
    Wgt[i-1] = GetWeight(col)

ev_df = pd.DataFrame( e_values )
ev_df.columns = [ 'SERM', 'Subject', 'Enviroment', 'Resource', 'Mechanism']
ev_df.set_index( df.columns[1:],  inplace=True)

CI_df = pd.DataFrame( CI )
CI_df.columns = [ 'SERM', 'Subject', 'Enviroment', 'Resource', 'Mechanism']
CI_df.set_index( df.columns[1:],  inplace=True)

Wgt_df = pd.DataFrame( Wgt )
Wgt_df.set_index( df.columns[1:],  inplace=True)
Wgt_df.columns = [ 'S', 'E', 'R', 'M', 'S1', 'S2', 'S3', 'S4', 'E1', 'E2', 'E3', 'E4', 'R1', 'R2', 'R3', 'R4', 'M1', 'M2', 'M3', 'M4']


from google.colab import drive
import shutil
drive.mount( '/content/drive')

dest_path = '/content/drive/MyDrive/result.xls'

# ExcelWriter 객체 생성
with pd.ExcelWriter(src_path, engine='xlsxwriter') as writer:
    ev_df.to_excel(writer, sheet_name='Eigen Values', index=True)

    CI_df.to_excel(writer, sheet_name='CI', index=True)

    Wgt_df.to_excel(writer, sheet_name='Weight', index=True)

shutil.copy( src_path , dest_path )


Saving answers.xlsx to answers (24).xlsx
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


'/content/drive/MyDrive/result.xls'